# Rock-Paper-Scissors

In [1]:
import os

import tensorflow as tf
import keras

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam, Nadam, Adadelta

Using TensorFlow backend.


In [2]:
training_data_path = os.getcwd()+"/Image_data/rps-train"
validation_data_path = os.getcwd()+"/Image_data/rps-validate"

In [5]:
class MultiClassNeuralNetwork():
    
    
    def modelBuilding(self,
                     training_data_path,
                     validation_data_path,
                      batch_size,
                     epochs,
                     learning_rate):
        
        train_datagen = ImageDataGenerator(rescale=1./255)
        
        train_generator = train_datagen.flow_from_directory(training_data_path,
                                                           target_size=(150,150),
                                                           batch_size=batch_size,
                                                           class_mode='categorical')
        
        validation_datagen = ImageDataGenerator(rescale=1./255)
        
        validation_generator = validation_datagen.flow_from_directory(validation_data_path,
                                                                    target_size=(150,150),
                                                                    batch_size=batch_size,
                                                                    class_mode='categorical')
        
        
        model = Sequential([
            tf.keras.layers.Conv2D(16, (3, 3),activation='relu',input_shape=(150, 150, 3)),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D(2, 2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])
        
        
        model.compile(optimizer=Adam(),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        
        
        model.summary()
        
        
        training_history = model.fit_generator(train_generator,
                            validation_data=validation_generator,
                            epochs=epochs,
                            verbose=1,
                            #callbacks=[callback])
                                              )
        
        return training_history, model
    
    
    def __init__(self):
        pass
    

In [6]:
neuralNet = MultiClassNeuralNetwork()

'''
training_data_path,
validation_data_path,
batch_size,
epochs,
learning_rate
'''

history, model = neuralNet.modelBuilding(training_data_path,
                                    validation_data_path,
                                    100,
                                    25,
                                    0.01)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               21234176  
_____________________________________

ValueError: Shape mismatch: The shape of labels (received (300,)) should equal the shape of logits except for the last dimension (received (100, 3)).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.figure(figsize=(17,10))
plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.plot(epochs, loss, 'g', label='Training loss')
loss
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.legend(loc=0)

plt.show()